# PEtab import

This illustrates how the PEtab format can be used together with the ODE simulation toolbox AMICI to define ODE based parameter estimation problems for pyABC.

To use this functionality, you need to have (at least) PEtab and AMICI installed. You can obtain these by installing pyABC with

    pip install pyabc[amici-petab]
    
or installing them manually via

    pip install petab amici

In [1]:
import petab
import pyabc
import amici.petab_import
from pyabc.petab import AmiciPetabImporter

In [2]:
petab_problem = petab.Problem.from_yaml("/home/yannik/benchmark-models/"
                                        "hackathon_contributions_new_data_format/"
                                        "Boehm_JProteomeRes2014/Boehm_JProteomeRes2014.yaml")
model = amici.petab_import.import_petab_problem(petab_problem)
solver = model.getSolver()

importer = AmiciPetabImporter(petab_problem, model, solver)

prior = importer.create_prior()
model = importer.create_model()
kernel = importer.create_kernel()

model(petab_problem.x_nominal_free_scaled)

{'llh': -138.22199570334107}

In [3]:
# this takes some time

abc = pyabc.ABCSMC(model, prior, kernel, 
                   eps=pyabc.Temperature(),
                   acceptor=pyabc.StochasticAcceptor())
abc.new(pyabc.storage.create_sqlite_db_id(), {})
abc.run()

INFO:History:Start <ABCSMC(id=11, start_time=2020-02-12 11:16:09.559777, end_time=None)>
INFO:ABC:Calibration sample before t=0.
INFO:ABC:t: 0, eps: 318454030.4305709.
INFO:ABC:Acceptance rate: 100 / 285 = 3.5088e-01, ESS=1.0000e+02.
INFO:ABC:t: 1, eps: 22569.1425095756.
Process Process-10:
Process Process-9:
Process Process-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process Process-11:
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.

  File "/home/yannik/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 3020, in _getitem_bool_array
    return self.take(indexer, axis=0)
  File "/home/yannik/anaconda3/lib/python3.7/site-packages/pandas/core/series.py", line 311, in __init__
    data = sanitize_array(data, index, dtype, copy, raise_cast_failure=True)
  File "/home/yannik/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 486, in __init__
    NDFrame.__init__(self, mgr, fastpath=True)
  File "/home/yannik/petab/petab/core.py", line 76, in get_notnull_columns
    return [col for col in candidates
  File "/home/yannik/anaconda3/lib/python3.7/site-packages/pandas/core/internals/construction.py", line 745, in sanitize_array
    not (is_extension_array_dtype(subarr.dtype) or is_extension_array_dtype(dtype))
  File "/home/yannik/anaconda3/lib/python3.7/site-packages/pandas/core/dtypes/common.py", line 1789, in is_extension_array_dtype
    return isinstance(dtype, ExtensionDtype) or registry.fin

KeyboardInterrupt: 